In [2]:
import os, sys, shutil, subprocess
import scipy
from scipy import ndimage
from scipy.spatial import distance

import numpy as np
import pandas as pd

from sklearn.decomposition import PCA

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
matplotlib.rcParams.update({'figure.max_open_warning': 0})
matplotlib.rcParams.update({'font.size': 12})

path     = os.getcwd()
pathrf   = path+'/opt_refframe'
pathpdb  = path+'/opt_pdb'
pathpar  = path+'/opt_par'
pathbps  = path+'/opt_det-seqs'

%matplotlib inline
path = os.getcwd()

In [4]:
Ncirc = 150
ematrix_dict = {0:'tilt', 1:'roll', 2:'twist', 3:'shift', 4:'slide', 5:'rise'}
incons        = ['or150','pc140','pc143','pc145','pc150','pc155']

seqs          = ['col05','col06','col07','col08','col09']
inseqs        = ['$A_1G_1$','$A_2G_2$','$A_3G_3$','$A_4G_4$','$A_5G_5$']
inseqs_labs   = {seqs[i]:inseqs1[i] for i in range(0, len(seqs))}
inseqs_colors = {'col05':'#00A5CF','col06':'#FB3640','col07':'#44AF69','col08':'#F0A202','col09':'#004385'}

ff_ind         = ['ideal','kabsch','olson']
ff_ind_colors  = {'kabsch':'#4682B4','ideal':'#B4464B','olson':'#B4AF46'}
ff_ind_marks   = {'ideal':'.','kabsch':'^','olson':'d'}


#--- Moving function ---------------------------------------------------------------------------------
def fig_move(currdir):
    for filename in os.listdir(currdir):
        if filename.endswith(".png"):
            shutil.move(currdir+'/'+filename, currdir+"/figures/"+filename)
    return


In [5]:
col150_df = pd.read_csv("data_col150_set04", index_col=0)

# Standard Circles: Energy Only

In [ ]:

for ff in ['ideal']:

    fig, axes = plt.subplots(nrows=1, ncols=len(), figsize=(10,4), sharey=True)


    df = col150_df.loc[(col150_df.seq_type=='std')&(col150_df.incon=='pc14')&(col150_df.forcefield==ff)].reset_index().set_index('seq')            
    df.plot.bar(y=GP, ylim=(globalprops_lims[GP][0], globalprops_lims[GP][1]),
               color=colors_dict['pc14'], 
               edgecolor='black', width=0.90, legend=False,
               ax=ax1)

    ax1.set_ylabel(globalprops_labs[GP], fontsize=20)
    ax1.set_xlabel('')
    ax1.minorticks_on()
    ax1.set_xticklabels([inseqs_labs[i] for i in set01], rotation=0)

    ax1.axhline(y=globalprops_horline['pc14'][GP], linestyle='-.', color=colors_dict['ideal'])
    for tick in ax1.xaxis.get_major_ticks():
        tick.label.set_fontsize(12)
    for tick in ax1.yaxis.get_major_ticks():
        tick.label.set_fontsize(14)

    del df

    df = col150_df.loc[(col150_df.seq_type=='std')&(col150_df.incon=='pc15')&(col150_df.forcefield==ff)].reset_index().set_index('seq')            
    df.plot.bar(y='eopt',
               color=colors_dict['pc15'], 
               edgecolor='black', width=0.90, legend=False,
               ax=ax2)

    ax2.set_xlabel('')
    ax2.minorticks_on()
    ax2.set_xticklabels([inseqs_labs[i] for i in set01], rotation=0)

    for tick in ax2.xaxis.get_major_ticks():
        tick.label.set_fontsize(12)
    for tick in ax2.yaxis.get_major_ticks():
        tick.label.set_fontsize(14)
    del df

    df = col150_df.loc[(col150_df.seq_type=='std')&(col150_df.incon=='or15')&(col150_df.forcefield==ff)].reset_index().set_index('seq')            
    df.plot.bar(y='eopt',
               color=colors_dict['or15'], 
               edgecolor='black', width=0.90, legend=False,
               ax=ax3)

    ax3.set_xlabel('')
    ax3.minorticks_on()
    ax3.set_xticklabels([inseqs_labs[i] for i in set01], rotation=0)

    for tick in ax3.xaxis.get_major_ticks():
        tick.label.set_fontsize(12)
    for tick in ax3.yaxis.get_major_ticks():
        tick.label.set_fontsize(14)
    del df


    plt.xticks(fontsize=12)
    plt.yticks(fontsize=14)
    plt.tight_layout()
    plt.show()
    #plt.savefig('set01_'+GP+'_'+incon+'_'+ff, dpi=300)
    plt.clf()

        

In [ ]:
# Try: 1 plot, group by sequence
df = col150_df.copy()

for forcefield in ['kabsch','olson']:
    df2 = df.loc[df.forcefield==forcefield]
    
    pc14 = []
    pc15 = []
    or15 = []

    for seq in set01:
        
        pc14.append(float(df2.loc[(df2.seq==seq)&(df2.incon=='pc14')].eopt))
        pc15.append(float(df2.loc[(df2.seq==seq)&(df2.incon=='pc15')].eopt))
        or15.append(float(df2.loc[(df2.seq==seq)&(df2.incon=='or15')].eopt))
    
    width=0.30
    x = np.arange(len(set01))
    
    fig, ax = plt.subplots(figsize=(8,4))
    
    rects1 = ax.bar(x-width, pc14, width, label='pc14', color=colors_dict['pc14'], edgecolor='black')
    rects2 = ax.bar(x      , pc15, width, label='pc15', color=colors_dict['pc15'], edgecolor='black')
    rects3 = ax.bar(x+width, or15, width, label='or15', color=colors_dict['or15'], edgecolor='black')
    
    ax.set_ylabel(globalprops_labs['eopt'], fontsize=16)
    ax.set_xticks(x)
    ax.set_xticklabels([inseqs_labs[i] for i in set01], fontsize=16)
    
    plt.ylim(0, 50)
    for tick in ax.yaxis.get_major_ticks():
        tick.label.set_fontsize(14)
    
    plt.show()
    plt.clf()
    del pc14, pc15, or15
    del df2
    print('-'*25)

del df


In [ ]:
# Try: 1 plot, group by sequence
df = col150_df.copy()

for forcefield in ['kabsch']:
    df2 = df.loc[df.forcefield==forcefield]
    
    pc14b = []
    pc14t = []
    
    pc15b = []
    pc15t = []
    
    or15b = []
    or15t = []

    for seq in set01:
        a = float(df2.loc[(df2.seq==seq)&(df2.incon=='pc14')]['eopt-tilt'])
        b = float(df2.loc[(df2.seq==seq)&(df2.incon=='pc14')]['eopt-roll'])
        pc14b.append(a+b)
        a = float(df2.loc[(df2.seq==seq)&(df2.incon=='pc15')]['eopt-tilt'])
        b = float(df2.loc[(df2.seq==seq)&(df2.incon=='pc15')]['eopt-roll'])
        pc15b.append(a+b)
        a = float(df2.loc[(df2.seq==seq)&(df2.incon=='or15')]['eopt-tilt'])
        b = float(df2.loc[(df2.seq==seq)&(df2.incon=='or15')]['eopt-roll'])
        or15b.append(a+b)
        del a,b
        
        pc14t.append(float(df2.loc[(df2.seq==seq)&(df2.incon=='pc14')]['eopt-twist']))
        pc15t.append(float(df2.loc[(df2.seq==seq)&(df2.incon=='pc15')]['eopt-twist']))
        or15t.append(float(df2.loc[(df2.seq==seq)&(df2.incon=='or15')]['eopt-twist']))
    
    width=0.30
    x = np.arange(len(set01))
    
    fig, ax = plt.subplots(figsize=(8,4))
    
    ax.bar(x-width, pc14b, width, label='pc14', color=colors_dict['pc14'], edgecolor='black')
    ax.bar(x      , pc15b, width, label='pc15', color=colors_dict['pc15'], edgecolor='black')
    ax.bar(x+width, or15b, width, label='or15', color=colors_dict['or15'], edgecolor='black')
    
    ax.bar(x-width, pc14t, width, color='lightgrey', edgecolor='black', bottom = pc14b)
    ax.bar(x      , pc15t, width, color='lightgrey', edgecolor='black', bottom = pc15b)
    ax.bar(x+width, or15t, width, color='lightgrey', edgecolor='black', bottom = or15b)
    
    ax.set_ylabel(globalprops_labs['eopt'], fontsize=16)
    ax.set_xticks(x)
    ax.set_xticklabels([inseqs_labs[i] for i in set01], fontsize=16)
    
    plt.ylim(0, 50)
    for tick in ax.yaxis.get_major_ticks():
        tick.label.set_fontsize(14)
    
    plt.legend(loc=1)
    
    plt.show()
    plt.clf()
    del pc14b, pc15b, or15b, pc14t, pc15t, or15t
    del df2
    print('-'*25)

del df

In [ ]:
print(pc14b)
print()
print(pc15b)
print()
print(or15b)
print()

In [ ]:
col150_df.head()

# Standard Circles

In [ ]:
globalprops      = ['eopt','Lk','Wr','Rg']

for GP in globalprops:
    for incon in incons:
        for ff in ['olson']:#['kabsch','olson']:
            
            df = col150_df.loc[(col150_df.seq_type=='std')&(col150_df.incon==incon)&(col150_df.forcefield==ff)].reset_index().set_index('seq')            
            fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5,5))
            
            df.plot.bar(y=GP, ylim=(globalprops_lims[GP][0], globalprops_lims[GP][1]),
                       color=colors_dict[incon], 
                       edgecolor='black', width=0.90, legend=False,
                       ax=ax)

            ax.set_ylabel(globalprops_labs[GP], fontsize=14)
            ax.set_xlabel('')
            ax.minorticks_on()
            ax.set_xticklabels([inseqs_labs[i] for i in set01], rotation=60)
            
            ax.axhline(y=globalprops_horline[incon][GP], linestyle='-.', color='black')
            
            plt.xticks(fontsize=12)
            plt.yticks(fontsize=12)
            plt.tight_layout()
            plt.show()
            #plt.savefig('set01_'+GP+'_'+incon+'_'+ff, dpi=300)
            plt.clf()
            
            del df
fig_move(path)


In [ ]:
set01b       = ['ideal']+set01
inseqs_labsb = {'ideal':'B-DNA', 'col05':'$A_1G_1$','col06':'$A_2G_2$','col07':'$A_3G_3$','col08':'$A_4G_4$','col09':'$A_5G_5$'}

idf = col150_df[col150_df.forcefield=='ideal']
'''
for ff in ['kabsch']:#,'olson']:
    fdf = col150_df[col150_df.forcefield==ff]
    for incon in incons:
        idf1 = idf[(idf.incon==incon)&(idf.seq_type=='std')].reset_index().set_index('seq')
        kdf1 = fdf[(fdf.incon==incon)&(fdf.seq_type=='std')].reset_index().set_index('seq')
        idf1['eopt-bend'] = idf1['eopt-tilt'] + idf1['eopt-roll']
        kdf1['eopt-bend'] = kdf1['eopt-tilt'] + kdf1['eopt-roll']
        
        kdf1 = kdf1.append(pd.Series(name='ideal'))
        kdf1.loc['ideal','eopt']       = idf1.loc['col05','eopt']
        kdf1.loc['ideal','eopt-bend']  = idf1.loc['col05','eopt-bend']
        kdf1.loc['ideal','eopt-twist'] = idf1.loc['col05','eopt-twist']
        
        kdf1 = kdf1.reindex(set01b)
        
        fig, (ax1) = plt.subplots(nrows=1, ncols=1, figsize=(6, 5))
        kdf1.plot.bar(stacked=True, 
                      y=['eopt-bend','eopt-twist'], 
                      ylim=(0, 50),
                      color=[[colors_dict["ideal"],colors_dict[incon],colors_dict[incon],colors_dict[incon],colors_dict[incon],colors_dict[incon]], 'lightgrey'], 
                      width=0.85,
                      edgecolor='black',
                      legend=False, 
                      ax=ax1)

        ax1.minorticks_on()
        ax1.set_xticklabels([inseqs_labsb[s] for s in set01b], rotation=60)
        ax1.set_ylabel('Optimized Energy ($k_B$T)', fontsize=14)
        ax1.set_xlabel('')

        plt.xticks(fontsize=12)
        plt.yticks(fontsize=12)
        plt.show()
        #plt.savefig('set01_eopt-stacked_'+incon+'_'+ff, dpi=300)
        plt.clf
        del idf1, kdf1
        
    del fdf
'''
for ff in ['olson']:#,'kabsch']:
    fdf = col150_df[col150_df.forcefield==ff]
    for incon in incons:
        idf1 = idf[(idf.incon==incon)&(idf.seq_type=='std')].reset_index().set_index('seq')
        kdf1 = fdf[(fdf.incon==incon)&(fdf.seq_type=='std')].reset_index().set_index('seq')
        idf1['eopt-bend'] = idf1['eopt-tilt'] + idf1['eopt-roll']
        kdf1['eopt-bend'] = kdf1['eopt-tilt'] + kdf1['eopt-roll']
        
        kdf1 = kdf1.append(pd.Series(name='ideal'))
        kdf1.loc['ideal','eopt']       = idf1.loc['col05','eopt']
        kdf1.loc['ideal','eopt-bend']  = idf1.loc['col05','eopt-bend']
        kdf1.loc['ideal','eopt-twist'] = idf1.loc['col05','eopt-twist']
        
        kdf1 = kdf1.reindex(set01b)
        
        fig, (ax1) = plt.subplots(nrows=1, ncols=1, figsize=(6, 5))
        kdf1.plot.bar(stacked=True, 
                      y=['eopt-bend','eopt-twist'], 
                      ylim=(0, 75),
                      color=[[colors_dict["ideal"],colors_dict[incon],colors_dict[incon],colors_dict[incon],colors_dict[incon],colors_dict[incon]],'lightgrey'], 
                      width=0.85,
                      edgecolor='black',
                      legend=False, 
                      ax=ax1)

        ax1.minorticks_on()
        ax1.set_xticklabels([inseqs_labsb[s] for s in set01b], rotation=60)
        ax1.set_ylabel('Optimized Energy ($k_B$T)', fontsize=14)
        ax1.set_xlabel('')

        plt.xticks(fontsize=12)
        plt.yticks(fontsize=12)
        plt.show()
        plt.savefig('set01_eopt-stacked_'+incon+'_'+ff, dpi=300)
        #plt.clf
        del idf1, kdf1
        
    del fdf
del idf, set01b, inseqs_labsb
fig_move(path)

In [ ]:
set01b       = ['ideal']+set01
inseqs_labsb = {'ideal':'B-DNA', 'col05':'$A_1G_1$','col06':'$A_2G_2$','col07':'$A_3G_3$','col08':'$A_4G_4$','col09':'$A_5G_5$'}

idf = col150_df[col150_df.forcefield=='ideal']

for ff in ['olson']:#,'kabsch']:
    fdf = col150_df[col150_df.forcefield==ff]
    for incon in incons:
        idf1 = idf[(idf.incon==incon)&(idf.seq_type=='std')].reset_index().set_index('seq')
        kdf1 = fdf[(fdf.incon==incon)&(fdf.seq_type=='std')].reset_index().set_index('seq')
        idf1['eopt-bend'] = idf1['eopt-tilt'] + idf1['eopt-roll']
        kdf1['eopt-bend'] = kdf1['eopt-tilt'] + kdf1['eopt-roll']
        idf1['eopt-shear'] = idf1['eopt-shift'] + idf1['eopt-slide']
        kdf1['eopt-shear'] = kdf1['eopt-shift'] + kdf1['eopt-slide']
        
        kdf1 = kdf1.append(pd.Series(name='ideal'))
        kdf1.loc['ideal','eopt']       = idf1.loc['col05','eopt']
        for x in ['tilt','roll','bend','twist','shift','slide','shear','rise']:
            kdf1.loc['ideal','eopt-'+x]  = idf1.loc['col05','eopt-'+x]
        
        kdf1 = kdf1.reindex(set01b)
        
        fig, (ax1) = plt.subplots(nrows=1, ncols=1, figsize=(3, 4))
        kdf1.plot.bar(stacked=True, 
                      y=['eopt-bend', 'eopt-twist','eopt-shear','eopt-rise'], 
                      ylim=(0, 75),
                      color=[[colors_dict["ideal"],colors_dict[incon],colors_dict[incon],colors_dict[incon],colors_dict[incon],colors_dict[incon]],'lightgrey','lightpink','lightblue'], 
                      width=0.85,
                      edgecolor='black',
                      legend=False, 
                      ax=ax1)

        ax1.minorticks_on()
        ax1.set_xticklabels([inseqs_labsb[s] for s in set01b], rotation=60)
        ax1.set_ylabel('Optimized Energy ($k_B$T)', fontsize=14)
        ax1.set_xlabel('')

        plt.xticks(fontsize=12)
        plt.yticks(fontsize=12)
        plt.show()
        #plt.savefig('set01_eopt-stacked_'+incon+'_'+ff, dpi=300)
        plt.clf
        del idf1, kdf1
        
    del fdf
del idf, set01b, inseqs_labsb
fig_move(path)

In [ ]:
set01b       = ['ideal']+set01
inseqs_labsb = {'ideal':'B-DNA', 'col05':'$A_1G_1$','col06':'$A_2G_2$','col07':'$A_3G_3$','col08':'$A_4G_4$','col09':'$A_5G_5$'}

fflimits1 = {'kabsch':[0, 50], 'olson':[0, 75]}
fflimits2 = {'kabsch':[0, 1.01], 'olson':[0, 2]}

idf = col150_df[col150_df.forcefield=='ideal']
for ff in ['olson']:#['kabsch','olson']:
    fdf = col150_df[col150_df.forcefield==ff]
    for incon in incons:
        idf1 = idf[(idf.incon==incon)&(idf.seq_type=='std')].reset_index().set_index('seq')
        kdf1 = fdf[(fdf.incon==incon)&(fdf.seq_type=='std')].reset_index().set_index('seq')
        
        idf1['eopt-bend'] = idf1['eopt-tilt'] + idf1['eopt-roll']
        kdf1['eopt-bend'] = kdf1['eopt-tilt'] + kdf1['eopt-roll']
        idf1['eopt-shear'] = idf1['eopt-shift'] + idf1['eopt-slide']
        kdf1['eopt-shear'] = kdf1['eopt-shift'] + kdf1['eopt-slide']
        
        kdf1 = kdf1.append(pd.Series(name='ideal'))
        kdf1.loc['ideal','eopt']       = idf1.loc['col05','eopt']
        for x in ['tilt','roll','bend','twist','shift','slide','shear','rise']:
            kdf1.loc['ideal','eopt-'+x]  = idf1.loc['col05','eopt-'+x]
        for x in ['tilt','roll','bend','twist','shift','slide','shear','rise']:
            kdf1['norm-'+x] = kdf1['eopt-'+x]/kdf1['eopt']

        del idf1
        kdf1 = kdf1.reindex(set01b)

        fig, (ax1,ax2) = plt.subplots(nrows=2, ncols=1, figsize=(6, 6), sharex=True)
        kdf1.plot.bar(stacked=True, 
                      y=['eopt-bend','eopt-twist'], 
                      ylim=(fflimits1[ff][0], fflimits1[ff][1]),
                      color=[[colors_dict["ideal"],colors_dict[incon],colors_dict[incon],colors_dict[incon],colors_dict[incon],colors_dict[incon]],'lightgrey'], 
                      width=0.85,
                      edgecolor='black',
                      legend=False, 
                      ax=ax1)
        ax1.minorticks_on()
        ax1.set_xticklabels([inseqs_labsb[s] for s in set01b], rotation=60)
        ax1.set_ylabel('Optimized Energy ($k_B$T)', fontsize=14)
        ax1.set_xlabel('')
        plt.xticks(fontsize=12)
        plt.yticks(fontsize=12)
        
        kdf1.plot.bar(stacked=True, 
                      y=['norm-bend','norm-twist'], 
                      ylim=(fflimits2[ff][0], fflimits2[ff][1]),
                      color=[[colors_dict["ideal"],colors_dict[incon],colors_dict[incon],colors_dict[incon],colors_dict[incon],colors_dict[incon]],'lightgrey'], 
                      width=0.85,
                      edgecolor='black',
                      legend=False, 
                      ax=ax2)
        plt.gca()
        ax2.minorticks_on()
        ax2.set_xticklabels([inseqs_labsb[s] for s in set01b], rotation=60)
        ax2.set_ylabel('Normalize Energy', fontsize=14)
        ax2.set_xlabel('')
        plt.xticks(fontsize=12)
        plt.yticks(fontsize=12)

        plt.tight_layout()
        plt.show()
        #plt.savefig('set01_eopt-dual-stacked1_'+incon+'_'+ff, dpi=300)
        plt.clf
        
        fig, (ax1,ax2) = plt.subplots(nrows=2, ncols=1, figsize=(6, 6), sharex=True)
        kdf1.plot.bar(stacked=True, 
                      y=['eopt-bend','eopt-twist', 'eopt-shear','eopt-rise'], 
                      ylim=(fflimits1[ff][0], fflimits1[ff][1]),
                      color=[[colors_dict["ideal"],colors_dict[incon],colors_dict[incon],colors_dict[incon],colors_dict[incon],colors_dict[incon]],'lightgrey','lightpink','lightblue'], 
                      width=0.85,
                      edgecolor='black',
                      legend=False, 
                      ax=ax1)
        ax1.minorticks_on()
        ax1.set_xticklabels([inseqs_labsb[s] for s in set01b], rotation=60)
        ax1.set_ylabel('Optimized Energy ($k_B$T)', fontsize=14)
        ax1.set_xlabel('')
        plt.xticks(fontsize=12)
        plt.yticks(fontsize=12)
        
        kdf1.plot.bar(stacked=True, 
                      y=['norm-bend','norm-twist', 'norm-shear','norm-rise'], 
                      ylim=(fflimits2[ff][0], fflimits2[ff][1]),
                      color=[[colors_dict["ideal"],colors_dict[incon],colors_dict[incon],colors_dict[incon],colors_dict[incon],colors_dict[incon]],'lightgrey','lightpink','lightblue'], 
                      width=0.85,
                      edgecolor='black',
                      legend=False, 
                      ax=ax2)
        plt.gca()
        ax2.minorticks_on()
        ax2.set_xticklabels([inseqs_labsb[s] for s in set01b], rotation=60)
        ax2.set_ylabel('Normalize Energy', fontsize=14)
        ax2.set_xlabel('')
        plt.xticks(fontsize=12)
        plt.yticks(fontsize=12)

        plt.tight_layout()
        plt.show()
        #plt.savefig('set01_eopt-dual-stacked2_'+incon+'_'+ff, dpi=300)
        plt.clf
        
        
        del kdf1

    del fdf

del idf, set01b, inseqs_labsb
fig_move(path)


In [ ]:
set01b       = ['ideal']+set01
inseqs_labsb = {'ideal':'B-DNA', 'col05':'$A_1G_1$','col06':'$A_2G_2$','col07':'$A_3G_3$','col08':'$A_4G_4$','col09':'$A_5G_5$'}

fflimits1 = {'kabsch':[0, 50], 'olson':[0, 75]}
fflimits2 = {'kabsch':[0, 1.01], 'olson':[0, 2]}

idf = col150_df[col150_df.forcefield=='ideal']
for ff in ['kabsch']:
    fdf = col150_df[col150_df.forcefield==ff]
    
    fig, ((ax1,ax3,ax5),(ax2,ax4,ax6)) = plt.subplots(nrows=2, ncols=3, figsize=(10, 6), sharex=True, gridspec_kw={'height_ratios':[4,2]})
    
    idf1 = idf[(idf.incon=="pc14")&(idf.seq_type=='std')].reset_index().set_index('seq')
    kdf1 = fdf[(fdf.incon=="pc14")&(fdf.seq_type=='std')].reset_index().set_index('seq')
    idf1['eopt-bend'] = idf1['eopt-tilt'] + idf1['eopt-roll']
    kdf1['eopt-bend'] = kdf1['eopt-tilt'] + kdf1['eopt-roll']
    idf1['eopt-shear'] = idf1['eopt-shift'] + idf1['eopt-slide']
    kdf1['eopt-shear'] = kdf1['eopt-shift'] + kdf1['eopt-slide']
    kdf1 = kdf1.append(pd.Series(name='ideal'))
    kdf1.loc['ideal','eopt']       = idf1.loc['col05','eopt']
    for x in ['tilt','roll','bend','twist','shift','slide','shear','rise']:
        kdf1.loc['ideal','eopt-'+x]  = idf1.loc['col05','eopt-'+x]
    for x in ['tilt','roll','bend','twist','shift','slide','shear','rise']:
        kdf1['norm-'+x] = kdf1['eopt-'+x]/kdf1['eopt']
    kdf1 = kdf1.reindex(set01b)
    kdf1.plot.bar(stacked=True, 
                  y=['eopt-bend','eopt-twist'], 
                  ylim=(fflimits1[ff][0], fflimits1[ff][1]),
                  color=[[colors_dict['ideal'], colors_dict['pc14'], colors_dict['pc14'],colors_dict['pc14'],colors_dict['pc14'],colors_dict['pc14']],'lightgrey'], 
                  width=0.85,
                  edgecolor='black',
                  legend=False, 
                  ax=ax1)
    ax1.minorticks_on()
    ax1.set_xticklabels([inseqs_labsb[s] for s in set01b], rotation=60, fontsize=12)
    ax1.set_ylabel('Optimized\nEnergy ($k_B$T)', fontsize=18)
    for tick in ax1.yaxis.get_major_ticks():
        tick.label.set_fontsize(14)
    ax1.set_xlabel('')
    kdf1.plot.bar(stacked=True, 
                  y=['norm-bend','norm-twist'], 
                  ylim=(fflimits2[ff][0], fflimits2[ff][1]),
                  color=[[colors_dict['ideal'], colors_dict['pc14'], colors_dict['pc14'],colors_dict['pc14'],colors_dict['pc14'],colors_dict['pc14']],'lightgrey'], 
                  width=0.85,
                  edgecolor='black',
                  legend=False, 
                  ax=ax2)
    ax2.minorticks_on()
    ax2.set_xticklabels([inseqs_labsb[s] for s in set01b], rotation=60, fontsize=12)
    ax2.set_ylabel('Normalized\nEnergy', fontsize=18)
    for tick in ax2.yaxis.get_major_ticks():
        tick.label.set_fontsize(10)
    ax2.set_xlabel('')
    del idf1,kdf1
    
    idf1 = idf[(idf.incon=="pc15")&(idf.seq_type=='std')].reset_index().set_index('seq')
    kdf1 = fdf[(fdf.incon=="pc15")&(fdf.seq_type=='std')].reset_index().set_index('seq')
    idf1['eopt-bend'] = idf1['eopt-tilt'] + idf1['eopt-roll']
    kdf1['eopt-bend'] = kdf1['eopt-tilt'] + kdf1['eopt-roll']
    idf1['eopt-shear'] = idf1['eopt-shift'] + idf1['eopt-slide']
    kdf1['eopt-shear'] = kdf1['eopt-shift'] + kdf1['eopt-slide']
    kdf1 = kdf1.append(pd.Series(name='ideal'))
    kdf1.loc['ideal','eopt']       = idf1.loc['col05','eopt']
    for x in ['tilt','roll','bend','twist','shift','slide','shear','rise']:
        kdf1.loc['ideal','eopt-'+x]  = idf1.loc['col05','eopt-'+x]
    for x in ['tilt','roll','bend','twist','shift','slide','shear','rise']:
        kdf1['norm-'+x] = kdf1['eopt-'+x]/kdf1['eopt']
    kdf1 = kdf1.reindex(set01b)
    kdf1.plot.bar(stacked=True, 
                  y=['eopt-bend','eopt-twist'], 
                  ylim=(fflimits1[ff][0], fflimits1[ff][1]),
                  color=[[colors_dict['ideal'], colors_dict['pc15'], colors_dict['pc15'],colors_dict['pc15'],colors_dict['pc15'],colors_dict['pc15']],'lightgrey'], 
                  width=0.85,
                  edgecolor='black',
                  legend=False, 
                  ax=ax3)
    ax3.minorticks_on()
    ax3.set_xticklabels([inseqs_labsb[s] for s in set01b], rotation=60)
    ax3.set_ylabel('')
    ax3.set_xlabel('')
    kdf1.plot.bar(stacked=True, 
                  y=['norm-bend','norm-twist'], 
                  ylim=(fflimits2[ff][0], fflimits2[ff][1]),
                  color=[[colors_dict['ideal'], colors_dict['pc15'], colors_dict['pc15'],colors_dict['pc15'],colors_dict['pc15'],colors_dict['pc15']],'lightgrey'], 
                  width=0.85,
                  edgecolor='black',
                  legend=False, 
                  ax=ax4)
    ax4.minorticks_on()
    ax4.set_xticklabels([inseqs_labsb[s] for s in set01b], rotation=60)
    ax4.set_ylabel('')
    for tick in ax4.yaxis.get_major_ticks():
        tick.label.set_fontsize(10)
    ax4.set_xlabel('')
    del idf1,kdf1
    
    idf1 = idf[(idf.incon=="or15")&(idf.seq_type=='std')].reset_index().set_index('seq')
    kdf1 = fdf[(fdf.incon=="or15")&(fdf.seq_type=='std')].reset_index().set_index('seq')
    idf1['eopt-bend'] = idf1['eopt-tilt'] + idf1['eopt-roll']
    kdf1['eopt-bend'] = kdf1['eopt-tilt'] + kdf1['eopt-roll']
    idf1['eopt-shear'] = idf1['eopt-shift'] + idf1['eopt-slide']
    kdf1['eopt-shear'] = kdf1['eopt-shift'] + kdf1['eopt-slide']
    kdf1 = kdf1.append(pd.Series(name='ideal'))
    kdf1.loc['ideal','eopt']       = idf1.loc['col05','eopt']
    for x in ['tilt','roll','bend','twist','shift','slide','shear','rise']:
        kdf1.loc['ideal','eopt-'+x]  = idf1.loc['col05','eopt-'+x]
    for x in ['tilt','roll','bend','twist','shift','slide','shear','rise']:
        kdf1['norm-'+x] = kdf1['eopt-'+x]/kdf1['eopt']
    kdf1 = kdf1.reindex(set01b)
    kdf1.plot.bar(stacked=True, 
                  y=['eopt-bend','eopt-twist'], 
                  ylim=(fflimits1[ff][0], fflimits1[ff][1]),
                  color=[[colors_dict['ideal'], colors_dict['or15'], colors_dict['or15'],colors_dict['or15'],colors_dict['or15'],colors_dict['or15']],'lightgrey'], 
                  width=0.85,
                  edgecolor='black',
                  legend=False, 
                  ax=ax5)
    ax5.minorticks_on()
    ax5.set_xticklabels([inseqs_labsb[s] for s in set01b], rotation=60)
    ax5.set_ylabel('')
    ax5.set_xlabel('')
    kdf1.plot.bar(stacked=True, 
                  y=['norm-bend','norm-twist'], 
                  ylim=(fflimits2[ff][0], fflimits2[ff][1]),
                  color=[[colors_dict['ideal'], colors_dict['or15'], colors_dict['or15'],colors_dict['or15'],colors_dict['or15'],colors_dict['or15']],'lightgrey'], 
                  width=0.85,
                  edgecolor='black',
                  legend=False, 
                  ax=ax6)
    ax6.minorticks_on()
    ax6.set_xticklabels([inseqs_labsb[s] for s in set01b], rotation=60)
    ax6.set_ylabel('')
    for tick in ax6.yaxis.get_major_ticks():
        tick.label.set_fontsize(10)
    ax6.set_xlabel('')
    del idf1,kdf1
        
    
    plt.tight_layout()
    plt.show()
    #plt.savefig('set01_eopt-dual-stacked1_'+incon+'_'+ff, dpi=300)
    plt.clf
        
        
del idf, set01b, inseqs_labsb
fig_move(path)


In [ ]:
globalprops      = ['eopt','Lk','Wr','Rg']

for i in range(1, len(globalprops)):
    for incon in incons:
        df0 = col150_df.loc[(col150_df.incon==incon)&(col150_df.forcefield=='kabsch')].reset_index().set_index('seq')

        fig, (ax0) = plt.subplots(nrows=1, ncols=1, figsize=(6,5))

        df0.plot.scatter(y=globalprops[0], ylim=(globalprops_lims[globalprops[0]][0], globalprops_lims[globalprops[0]][1]),
                         x=globalprops[i], xlim=(globalprops_lims[globalprops[i]][0], globalprops_lims[globalprops[i]][1]), 
                         color=[colors_dict[x] for x in set01], alpha=0.50, 
                         edgecolor='black', s=75, marker='s',
                         ax=ax0)

        for ax in [ax0]:
            ax.minorticks_on()
            ax.set_xlabel(globalprops_labs[globalprops[i]], fontsize=14)
            ax.set_ylabel(globalprops_labs[globalprops[0]], fontsize=14)
            ax.grid('on', linestyle=':', linewidth=0.5)
            ax.tick_params(axis='both', labelsize='12')

        for ax in [ax0]:
            ax.axvline(x=globalprops_horline[incon][globalprops[i]], linestyle=':', color=colors_dict[incon])
            ax.axhline(y=globalprops_horline[incon][globalprops[0]], linestyle=':', color=colors_dict[incon])
        plt.tight_layout()
        plt.show()
        #plt.savefig('set01_scatter-'+globalprops[0]+'-'+globalprops[i]+'_'+incon+'_kabsch', dpi=300)
        plt.clf()

        del df0

for incon in incons:
    df0 = col150_df.loc[(col150_df.incon==incon)&(col150_df.forcefield=='kabsch')].reset_index().set_index('seq')
    fig, (ax0) = plt.subplots(nrows=1, ncols=1, figsize=(6,5))

    df0.plot.scatter(y=globalprops[2], ylim=(globalprops_lims[globalprops[2]][0], globalprops_lims[globalprops[2]][1]),
                     x=globalprops[3], xlim=(globalprops_lims[globalprops[3]][0], globalprops_lims[globalprops[3]][1]), 
                     color=[colors_dict[x] for x in set01], alpha=0.50, 
                     edgecolor='black', s=75, marker='s',
                     ax=ax0)

    for ax in [ax0]:
        ax.minorticks_on()
        ax.set_xlabel(globalprops_labs[globalprops[3]], fontsize=14)
        ax.set_ylabel(globalprops_labs[globalprops[2]], fontsize=14)
        ax.grid('on', linestyle=':', linewidth=0.5)
        ax.tick_params(axis='both', labelsize='12')

    for ax in [ax0]:
        ax.axvline(x=globalprops_horline[incon][globalprops[3]], linestyle=':', color=colors_dict[incon])
        ax.axhline(y=globalprops_horline[incon][globalprops[2]], linestyle=':', color=colors_dict[incon])
    plt.tight_layout()
    plt.show()
    #plt.savefig('set01_scatter-'+globalprops[2]+'-'+globalprops[3]+'_'+incon+'_kabsch', dpi=300)
    plt.clf()

    del df0
fig_move(path)

In [ ]:
globalprops      = ['eopt','Lk','Wr','Rg']

for i in range(1, len(globalprops)):
    for incon in incons:
        df0 = col150_df.loc[(col150_df.incon==incon)&(col150_df.forcefield=='olson')].reset_index().set_index('seq')

        fig, (ax0) = plt.subplots(nrows=1, ncols=1, figsize=(6,5))

        df0.plot.scatter(y=globalprops[0], ylim=(globalprops_lims[globalprops[0]][0], globalprops_lims[globalprops[0]][1]),
                         x=globalprops[i], xlim=(globalprops_lims[globalprops[i]][0], globalprops_lims[globalprops[i]][1]), 
                         color=[colors_dict[x] for x in set01], alpha=0.50, 
                         edgecolor='black', s=75, marker='s',
                         ax=ax0)

        for ax in [ax0]:
            ax.minorticks_on()
            ax.set_xlabel(globalprops_labs[globalprops[i]], fontsize=14)
            ax.set_ylabel(globalprops_labs[globalprops[0]], fontsize=14)
            ax.grid('on', linestyle=':', linewidth=0.5)
            ax.tick_params(axis='both', labelsize='12')

        for ax in [ax0]:
            ax.axvline(x=globalprops_horline[incon][globalprops[i]], linestyle=':', color=colors_dict[incon])
            ax.axhline(y=globalprops_horline[incon][globalprops[0]], linestyle=':', color=colors_dict[incon])
        plt.tight_layout()
        plt.show()
        #plt.savefig('set01_scatter-'+globalprops[0]+'-'+globalprops[i]+'_'+incon+'_olson', dpi=300)
        plt.clf()

        del df0

for incon in incons:
    df0 = col150_df.loc[(col150_df.incon==incon)&(col150_df.forcefield=='olson')].reset_index().set_index('seq')
    fig, (ax0) = plt.subplots(nrows=1, ncols=1, figsize=(6,5))

    df0.plot.scatter(y=globalprops[2], ylim=(globalprops_lims[globalprops[2]][0], globalprops_lims[globalprops[2]][1]),
                     x=globalprops[3], xlim=(globalprops_lims[globalprops[3]][0], globalprops_lims[globalprops[3]][1]), 
                     color=[colors_dict[x] for x in set01], alpha=0.50, 
                     edgecolor='black', s=75, marker='s',
                     ax=ax0)

    for ax in [ax0]:
        ax.minorticks_on()
        ax.set_xlabel(globalprops_labs[globalprops[3]], fontsize=14)
        ax.set_ylabel(globalprops_labs[globalprops[2]], fontsize=14)
        ax.grid('on', linestyle=':', linewidth=0.5)
        ax.tick_params(axis='both', labelsize='12')

    for ax in [ax0]:
        ax.axvline(x=globalprops_horline[incon][globalprops[3]], linestyle=':', color=colors_dict[incon])
        ax.axhline(y=globalprops_horline[incon][globalprops[2]], linestyle=':', color=colors_dict[incon])
    plt.tight_layout()
    plt.show()
    #plt.savefig('set01_scatter-'+globalprops[2]+'-'+globalprops[3]+'_'+incon+'_olson', dpi=300)
    plt.clf()

    del df0
fig_move(path)

In [ ]:
globalprops      = ['eopt','Lk','Wr','Rg']

for i in range(1, len(globalprops)):
    fig, (ax0) = plt.subplots(nrows=1, ncols=1, figsize=(6,5))
    
    for incon in incons:
        df0 = col150_df.loc[(col150_df.incon==incon)&(col150_df.forcefield=='kabsch')].reset_index().set_index('seq')
        df0.plot.scatter(y=globalprops[0], ylim=(globalprops_lims[globalprops[0]][0], globalprops_lims[globalprops[0]][1]),
                         x=globalprops[i], xlim=(globalprops_lims[globalprops[i]][0], globalprops_lims[globalprops[i]][1]), 
                         color=colors_dict[incon], alpha=0.50, 
                         edgecolor='black', s=100, marker=incons_markers[incon],
                         ax=ax0)

    for ax in [ax0]:
        ax.minorticks_on()
        ax.set_xlabel(globalprops_labs[globalprops[i]], fontsize=18)
        ax.set_ylabel(globalprops_labs[globalprops[0]], fontsize=18)
        ax.grid('on', linestyle=':', linewidth=0.5)
        ax.tick_params(axis='both', labelsize='14')
    
    plt.tight_layout()
    plt.show()
    #plt.savefig('set01_scatter-'+globalprops[0]+'-'+globalprops[i]+'_all_kabsch', dpi=300)
    plt.clf()

    del df0

    

fig, (ax0) = plt.subplots(nrows=1, ncols=1, figsize=(6,5))

for incon in incons:
    df0 = col150_df.loc[(col150_df.incon==incon)&(col150_df.forcefield=='kabsch')].reset_index().set_index('seq')
    df0.plot.scatter(x=globalprops[2], xlim=(globalprops_lims[globalprops[2]][0], globalprops_lims[globalprops[2]][1]),
                     y=globalprops[3], ylim=(globalprops_lims[globalprops[3]][0], globalprops_lims[globalprops[3]][1]), 
                     color=colors_dict[incon], alpha=0.50, 
                     edgecolor='black', s=100, marker=incons_markers[incon],
                     ax=ax0)

for ax in [ax0]:
    ax.minorticks_on()
    ax.set_ylabel(globalprops_labs[globalprops[3]], fontsize=18)
    ax.set_xlabel(globalprops_labs[globalprops[2]], fontsize=18)
    ax.grid('on', linestyle=':', linewidth=0.5)
    ax.tick_params(axis='both', labelsize='14')


plt.tight_layout()
plt.show()
#plt.savefig('set01_scatter-'+globalprops[2]+'-'+globalprops[3]+'_all_kabsch', dpi=300)
plt.clf()

del df0

In [ ]:
globalprops      = ['eopt','Lk','Wr','Rg']

for i in range(1, len(globalprops)):
    fig, (ax0) = plt.subplots(nrows=1, ncols=1, figsize=(6,5))
    
    for incon in incons:
        df0 = col150_df.loc[(col150_df.incon==incon)&(col150_df.forcefield=='olson')].reset_index().set_index('seq')
        df0.plot.scatter(y=globalprops[0], ylim=(globalprops_lims[globalprops[0]][0], globalprops_lims[globalprops[0]][1]),
                         x=globalprops[i], xlim=(globalprops_lims[globalprops[i]][0], globalprops_lims[globalprops[i]][1]), 
                         color=colors_dict[incon], alpha=0.50, 
                         edgecolor='black', s=100, marker=incons_markers[incon],
                         ax=ax0)

    for ax in [ax0]:
        ax.minorticks_on()
        ax.set_xlabel(globalprops_labs[globalprops[i]], fontsize=18)
        ax.set_ylabel(globalprops_labs[globalprops[0]], fontsize=18)
        ax.grid('on', linestyle=':', linewidth=0.5)
        ax.tick_params(axis='both', labelsize='14')
    
    plt.tight_layout()
    plt.show()
    #plt.savefig('set01_scatter-'+globalprops[0]+'-'+globalprops[i]+'_all_olson', dpi=300)
    plt.clf()

    del df0

    

fig, (ax0) = plt.subplots(nrows=1, ncols=1, figsize=(6,5))

for incon in incons:
    df0 = col150_df.loc[(col150_df.incon==incon)&(col150_df.forcefield=='olson')].reset_index().set_index('seq')
    df0.plot.scatter(x=globalprops[2], xlim=(globalprops_lims[globalprops[2]][0], globalprops_lims[globalprops[2]][1]),
                     y=globalprops[3], ylim=(globalprops_lims[globalprops[3]][0], globalprops_lims[globalprops[3]][1]), 
                     color=colors_dict[incon], alpha=0.50, 
                     edgecolor='black', s=100, marker=incons_markers[incon],
                     ax=ax0)

for ax in [ax0]:
    ax.minorticks_on()
    ax.set_ylabel(globalprops_labs[globalprops[3]], fontsize=18)
    ax.set_xlabel(globalprops_labs[globalprops[2]], fontsize=18)
    ax.grid('on', linestyle=':', linewidth=0.5)
    ax.tick_params(axis='both', labelsize='14')


plt.tight_layout()
plt.show()
#plt.savefig('set01_scatter-'+globalprops[2]+'-'+globalprops[3]+'_all_olson', dpi=300)
plt.clf()

del df0